# 🌐 Distributed Architecture - 분산 시스템 구성

이 노트북은 beanllm의 분산 아키텍처를 학습합니다. Redis와 Kafka를 활용하여 대규모 시스템을 구축할 수 있습니다.

## 📋 목차

- [1. 분산 아키텍처 기본](#1-분산-아키텍처-기본)
- [2. Redis 활용](#2-redis-활용)
- [3. Kafka 활용](#3-kafka-활용)
- [4. 데코레이터 패턴](#4-데코레이터-패턴)
- [5. 동적 설정](#5-동적-설정)
- [6. 실전 예제](#6-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ 분산 아키텍처의 개념을 이해할 수 있습니다
- ✅ Redis를 활용한 Rate Limiting과 Caching을 설정할 수 있습니다
- ✅ Kafka를 활용한 이벤트 스트리밍을 구성할 수 있습니다
- ✅ 데코레이터 패턴으로 분산 기능을 적용할 수 있습니다
- ✅ 동적으로 설정을 변경할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료
- Redis 및 Kafka 서버 실행 (선택적)


## 1. 분산 아키텍처 기본

분산 아키텍처는 Redis와 Kafka를 활용하여 대규모 시스템을 구축합니다.

> 💡 **분산 아키텍처의 핵심**:
> - **Redis**: Rate Limiting, Caching, Distributed Locks, Short-term Queues
> - **Kafka**: Event Streaming, Long-term Queues, Streaming Processing
> - **Fallback**: 분산 컴포넌트 실패 시 자동으로 인메모리로 전환
> - **데코레이터 패턴**: 코드 변경 없이 분산 기능 적용


In [ ]:
import os

# 분산 아키텍처 활성화
os.environ["USE_DISTRIBUTED"] = "true"
os.environ["REDIS_HOST"] = "localhost"
os.environ["REDIS_PORT"] = "6379"
os.environ["KAFKA_BOOTSTRAP_SERVERS"] = "localhost:9092"

from beanllm.infrastructure.distributed import (
    check_redis_health,
    check_kafka_health,
)

# 건강 상태 확인
redis_ok = check_redis_health()
kafka_ok = check_kafka_health()

print("✅ 분산 아키텍처 설정 완료!")
print(f"📊 Redis 상태: {'✅ 연결됨' if redis_ok else '❌ 연결 실패'}")
print(f"📊 Kafka 상태: {'✅ 연결됨' if kafka_ok else '❌ 연결 실패'}")

if not redis_ok or not kafka_ok:
    print("\n💡 Fallback 모드:")
    print("  - 분산 컴포넌트 실패 시 자동으로 인메모리로 전환")
    print("  - 기능은 정상 작동하지만 분산 기능은 비활성화")

## 2. Redis 활용

Redis는 Rate Limiting, Caching, Distributed Locks에 사용됩니다.

### 2.1 Rate Limiting

요청 속도를 제한하여 API 비용을 관리합니다.


In [ ]:
from beanllm.infrastructure.distributed import get_rate_limiter
import asyncio
import time

# Rate Limiter 가져오기
rate_limiter = get_rate_limiter()


# Rate Limit 확인
async def check_rate_limit():
    key = "user:123"
    limit = 5  # 5초당 5개 요청

    print("🔍 Rate Limiting 테스트:\n")

    for i in range(10):
        allowed = await rate_limiter.is_allowed(key, limit, window=5.0)

        if allowed:
            print(f"✅ 요청 {i+1}: 허용됨")
            # 실제 요청 처리 시뮬레이션
            await asyncio.sleep(0.5)
        else:
            print(f"❌ 요청 {i+1}: Rate Limit 초과 - 요청 거부")
            # 대기 시간 계산
            wait_time = await rate_limiter.get_wait_time(key, limit, window=5.0)
            print(f"   ⏳ 대기 시간: {wait_time:.2f}초")


# 실행
await check_rate_limit()

print("\n💡 Rate Limiting의 장점:")
print("  - API 비용 관리 (OpenAI, Anthropic 등)")
print("  - 서버 부하 방지")
print("  - 공정한 리소스 분배")
print("  - DDoS 공격 방어")

### 2.2 Caching

자주 사용되는 결과를 캐싱하여 성능을 향상시킵니다.


In [ ]:
from beanllm.infrastructure.distributed import get_cache
import hashlib
import time

# Cache 가져오기
cache = get_cache()


async def use_cache_example():
    """캐싱을 사용한 쿼리 처리"""

    print("🔍 Caching 테스트:\n")

    query = "Python이란?"
    key = f"rag:query:{hashlib.md5(query.encode()).hexdigest()}"

    # 첫 번째 요청 (캐시 미스)
    print("📝 첫 번째 요청:")
    start = time.time()

    cached = await cache.get(key)
    if cached:
        print(f"✅ 캐시 히트: {cached}")
    else:
        print("❌ 캐시 미스 - 실제 처리 수행")
        # 실제 처리 시뮬레이션 (1초 소요)
        await asyncio.sleep(1.0)
        result = "Python은 고급 프로그래밍 언어입니다. 간결하고 읽기 쉬운 문법이 특징입니다."

        # 캐시 저장 (TTL: 3600초)
        await cache.set(key, result, ttl=3600)
        print(f"✅ 캐시 저장: {key[:32]}...")
        print(f"📝 결과: {result[:50]}...")

    elapsed = time.time() - start
    print(f"⏱️  처리 시간: {elapsed:.3f}초\n")

    # 두 번째 요청 (캐시 히트)
    print("📝 두 번째 요청 (동일 쿼리):")
    start = time.time()

    cached = await cache.get(key)
    if cached:
        print(f"✅ 캐시 히트!")
        print(f"📝 결과: {cached[:50]}...")
    else:
        print("❌ 캐시 미스")

    elapsed = time.time() - start
    print(f"⏱️  처리 시간: {elapsed:.3f}초")

    # 성능 개선 계산
    improvement = (1.0 - elapsed) / 1.0 * 100
    print(f"\n📊 성능 개선: {improvement:.1f}% 빠름")


# 실행
await use_cache_example()

print("\n💡 Caching의 장점:")
print("  - 응답 시간 단축 (수 초 → 밀리초)")
print("  - API 비용 절감 (동일 쿼리 재사용)")
print("  - 서버 부하 감소")
print("  - 사용자 경험 개선")

## 3. Kafka 활용

Kafka는 이벤트 스트리밍과 장기 작업 큐에 사용됩니다.

### 3.1 이벤트 스트리밍

모든 요청과 동작을 이벤트로 기록합니다.


In [ ]:
from beanllm.infrastructure.distributed import get_event_logger
import json

# Event Logger 가져오기
event_logger = get_event_logger()


async def log_event_example():
    """이벤트 스트리밍 예제"""

    print("🔍 Event Streaming 테스트:\n")

    # 다양한 이벤트 발행
    events = [
        {
            "event_type": "llm.request",
            "data": {
                "model": "gpt-4o",
                "user_id": "user123",
                "tokens": 150,
                "cost": 0.005,
            },
        },
        {
            "event_type": "rag.query",
            "data": {
                "query": "Python이란?",
                "user_id": "user123",
                "top_k": 5,
                "latency": 0.234,
            },
        },
        {
            "event_type": "rag.retrieve",
            "data": {
                "query": "Python이란?",
                "num_docs": 5,
                "latency": 0.123,
            },
        },
        {
            "event_type": "llm.response",
            "data": {
                "model": "gpt-4o",
                "user_id": "user123",
                "tokens": 320,
                "latency": 1.456,
            },
        },
    ]

    for event in events:
        await event_logger.log_event(event_type=event["event_type"], data=event["data"])
        print(f"✅ 이벤트 발행: {event['event_type']}")
        print(f"   📝 데이터: {json.dumps(event['data'], indent=6, ensure_ascii=False)}")
        print()

    print("✅ 모든 이벤트 발행 완료!\n")


# 실행
await log_event_example()

print("💡 Event Streaming의 활용:")
print("  - 모든 요청 추적 (감사 로그)")
print("  - 분석 및 모니터링 (대시보드)")
print("  - 실시간 알림 (이상 감지)")
print("  - 데이터 분석 (사용 패턴)")
print("  - 디버깅 (문제 추적)")

print("\n💡 Kafka의 장점:")
print("  - 영구 저장 (데이터 손실 없음)")
print("  - 높은 처리량 (초당 수백만 메시지)")
print("  - 확장 가능 (수평 확장)")
print("  - 내구성 (복제)")

### 3.2 작업 큐

장기 작업을 큐에 넣어 비동기로 처리합니다.


In [ ]:
from beanllm.infrastructure.distributed import get_task_queue
import uuid

# Task Queue 가져오기
task_queue = get_task_queue()


async def queue_task_example():
    """작업 큐 예제"""

    print("🔍 Task Queue 테스트:\n")

    # 다양한 작업을 큐에 추가
    tasks = [
        {
            "task_type": "process_document",
            "payload": {"file_path": "document1.pdf", "user_id": "user123"},
            "priority": "high",
        },
        {
            "task_type": "process_document",
            "payload": {"file_path": "document2.pdf", "user_id": "user123"},
            "priority": "medium",
        },
        {
            "task_type": "generate_summary",
            "payload": {"doc_id": "doc123", "max_length": 500},
            "priority": "low",
        },
        {
            "task_type": "build_knowledge_graph",
            "payload": {"doc_ids": ["doc1", "doc2", "doc3"]},
            "priority": "high",
        },
    ]

    task_ids = []
    for task in tasks:
        task_id = await task_queue.enqueue(
            task_type=task["task_type"], payload=task["payload"], priority=task["priority"]
        )
        task_ids.append(task_id)
        print(f"✅ 작업 큐에 추가:")
        print(f"   📝 ID: {task_id}")
        print(f"   🏷️  타입: {task['task_type']}")
        print(f"   ⚡ 우선순위: {task['priority']}")
        print()

    print(f"✅ 총 {len(task_ids)}개 작업 큐에 추가 완료!\n")

    # 작업 처리 시뮬레이션 (별도 워커에서)
    print("🔄 작업 처리 시뮬레이션:\n")

    for i in range(2):
        # 우선순위가 높은 작업부터 처리
        task = await task_queue.dequeue("process_document")
        if task:
            print(f"⚙️  작업 처리 중: {task.id}")
            print(f"   📝 타입: {task.task_type}")
            print(f"   💾 페이로드: {task.payload}")

            # 작업 처리 시뮬레이션
            await asyncio.sleep(0.5)

            # 작업 완료
            await task_queue.complete(task.id)
            print(f"✅ 작업 완료: {task.id}\n")


# 실행
await queue_task_example()

print("💡 Task Queue의 활용:")
print("  - 장기 작업 비동기 처리")
print("  - 작업 우선순위 관리")
print("  - 작업 재시도 (실패 시)")
print("  - 작업 상태 추적")
print("  - 워커 스케일링")

print("\n💡 사용 사례:")
print("  - 대용량 문서 처리")
print("  - 배치 임베딩 생성")
print("  - Knowledge Graph 구축")
print("  - 정기 작업 스케줄링")

## 4. 데코레이터 패턴

데코레이터를 사용하여 코드 변경 없이 분산 기능을 적용할 수 있습니다.


In [ ]:
from beanllm.infrastructure.distributed.pipeline_decorators import (
    with_distributed_features,
)


# 데코레이터로 분산 기능 적용
@with_distributed_features(
    pipeline_type="rag",
    enable_cache=True,
    enable_rate_limiting=True,
    enable_event_streaming=True,
    cache_key_prefix="rag:query",
    rate_limit_key="rag:search",
    event_type="rag.query",
)
async def rag_query(query: str) -> str:
    """RAG 쿼리 (분산 기능 자동 적용)"""
    # 실제 로직만 작성
    # 분산 기능은 데코레이터가 자동 처리:
    #   1. Rate Limiting 확인
    #   2. 캐시 확인
    #   3. 실제 처리 (캐시 미스 시)
    #   4. 캐시 저장
    #   5. 이벤트 발행

    # 실제 RAG 처리 시뮬레이션
    print(f"🔍 쿼리 처리: {query}")
    await asyncio.sleep(0.5)  # 처리 시뮬레이션

    return f"답변: {query}에 대한 상세한 답변입니다. Python은 고급 프로그래밍 언어로..."


print("✅ 데코레이터 패턴 적용 완료!\n")

print("💡 데코레이터의 장점:")
print("  - 코드 변경 최소화 (3-5줄만 작성)")
print("  - 중복 코드 제거 (85-90% 감소)")
print("  - 일관된 분산 기능 적용")
print("  - 유지보수 용이")
print("  - 테스트 용이\n")

# 사용 예제
print("🔄 실행 예제:\n")

for i in range(3):
    result = await rag_query("Python이란?")
    print(f"📝 결과 {i+1}: {result[:50]}...\n")
    await asyncio.sleep(0.2)

print("💡 자동 적용된 기능:")
print("  ✅ Rate Limiting: 요청 속도 제한")
print("  ✅ Caching: 첫 번째 요청 후 캐시에서 반환")
print("  ✅ Event Streaming: 모든 요청 이벤트 발행")
print("  ✅ Fallback: 분산 컴포넌트 실패 시 인메모리로 전환")

## 5. 동적 설정

런타임에 분산 기능 설정을 변경할 수 있습니다.


In [ ]:
from beanllm.infrastructure.distributed import (
    update_pipeline_config,
    get_pipeline_config,
    reset_pipeline_config,
)
import json

print("🔍 동적 설정 변경 테스트:\n")

# 1. 초기 설정 확인
print("📊 초기 설정:")
config = get_pipeline_config("rag")
print(json.dumps(config, indent=2, ensure_ascii=False))
print()

# 2. Rate Limiting 비활성화
print("🔧 Rate Limiting 비활성화:")
update_pipeline_config(
    pipeline_type="rag",
    enable_rate_limiting=False,  # Rate Limiting 비활성화
)
config = get_pipeline_config("rag")
print(f"  enable_rate_limiting: {config['enable_rate_limiting']}")
print()

# 3. 캐시 TTL 변경
print("🔧 캐시 TTL 변경 (3600 → 7200초):")
update_pipeline_config(
    pipeline_type="rag",
    rag_cache_ttl=7200,  # 2시간으로 변경
)
config = get_pipeline_config("rag")
print(f"  rag_cache_ttl: {config['rag_cache_ttl']}초")
print()

# 4. 여러 설정 동시 변경
print("🔧 여러 설정 동시 변경:")
update_pipeline_config(
    pipeline_type="rag",
    enable_rate_limiting=True,
    enable_cache=True,
    enable_event_streaming=True,
    rag_cache_ttl=3600,
)
config = get_pipeline_config("rag")
print(f"  enable_rate_limiting: {config['enable_rate_limiting']}")
print(f"  enable_cache: {config['enable_cache']}")
print(f"  enable_event_streaming: {config['enable_event_streaming']}")
print(f"  rag_cache_ttl: {config['rag_cache_ttl']}초")
print()

# 5. 설정 초기화
print("🔧 설정 초기화:")
reset_pipeline_config("rag")
config = get_pipeline_config("rag")
print(f"  초기화 완료!")
print()

print("✅ 동적 설정 변경 완료!\n")

print("💡 동적 설정의 활용:")
print("  - 재시작 없이 설정 변경")
print("  - A/B 테스트 (사용자 그룹별 설정)")
print("  - 실시간 최적화 (성능 모니터링 기반)")
print("  - 긴급 대응 (문제 발생 시 빠른 비활성화)")
print("  - 비용 절감 (피크 시간 Rate Limit 조정)")

print("\n💡 사용 사례:")
print("  - 트래픽 급증 시 Rate Limit 강화")
print("  - 캐시 미스율 높을 때 TTL 증가")
print("  - 디버깅 시 Event Streaming 활성화")
print("  - 비용 절감 모드 (캐싱 강화)")

## 6. 실전 예제

### 6.1 분산 RAG 시스템

분산 아키텍처를 활용한 RAG 시스템입니다.


In [ ]:
from beanllm.infrastructure.distributed.pipeline_decorators import (
    with_distributed_features,
)
import time


# 분산 기능이 적용된 RAG
@with_distributed_features(
    pipeline_type="rag",
    enable_cache=True,
    enable_rate_limiting=True,
    enable_event_streaming=True,
    cache_key_prefix="rag:query",
    rate_limit_key="rag:search",
    event_type="rag.query",
)
async def distributed_rag_query(query: str, top_k: int = 5) -> dict:
    """분산 RAG 시스템"""

    # 실제 RAG 처리 시뮬레이션
    await asyncio.sleep(0.8)  # 검색 시뮬레이션

    # 결과 반환
    return {
        "query": query,
        "top_k": top_k,
        "results": [
            {
                "content": f"Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.",
                "score": 0.95,
                "source": "python_intro.pdf",
            },
            {
                "content": f"Python은 간결하고 읽기 쉬운 문법을 제공합니다.",
                "score": 0.89,
                "source": "python_features.pdf",
            },
            {
                "content": f"Python은 데이터 과학, 웹 개발, 자동화 등 다양한 분야에서 활용됩니다.",
                "score": 0.85,
                "source": "python_applications.pdf",
            },
        ],
        "answer": "Python은 고급 프로그래밍 언어로, 간결한 문법과 강력한 기능을 제공합니다.",
    }


print("🔍 분산 RAG 시스템 테스트:\n")

# 첫 번째 쿼리 (캐시 미스)
print("📝 첫 번째 쿼리:")
start = time.time()
result1 = await distributed_rag_query("Python이란?")
elapsed1 = time.time() - start
print(f"✅ 완료!")
print(f"   ⏱️  처리 시간: {elapsed1:.3f}초")
print(f"   📊 결과 수: {len(result1['results'])}")
print(f"   💬 답변: {result1['answer'][:50]}...")
print()

# 두 번째 쿼리 (캐시 히트)
print("📝 두 번째 쿼리 (동일):")
start = time.time()
result2 = await distributed_rag_query("Python이란?")
elapsed2 = time.time() - start
print(f"✅ 완료!")
print(f"   ⏱️  처리 시간: {elapsed2:.3f}초 (캐시 히트)")
print(f"   📊 결과 수: {len(result2['results'])}")
print()

# 성능 개선 계산
improvement = (elapsed1 - elapsed2) / elapsed1 * 100
print(f"📊 성능 개선: {improvement:.1f}% 빠름\n")

print("✅ 분산 RAG 시스템 완료!\n")

print("💡 자동 적용되는 기능:")
print("  ✅ Caching: 동일 쿼리 재사용 → 응답 시간 단축")
print("  ✅ Rate Limiting: API 비용 관리 → 비용 절감")
print("  ✅ Event Streaming: 모든 요청 추적 → 분석 & 모니터링")
print("  ✅ Distributed Locks: 동시성 제어 → 데이터 일관성")
print("  ✅ Fallback: 실패 시 자동 전환 → 안정성")

print("\n💡 실전 활용:")
print("  - 고객 지원 챗봇 (캐싱으로 응답 시간 단축)")
print("  - 문서 검색 시스템 (Rate Limiting으로 비용 관리)")
print("  - 분석 대시보드 (Event Streaming으로 실시간 모니터링)")

## 📸 시각화 예제

### 분산 아키텍처 구조

> 💡 **이미지 필요**: 분산 아키텍처의 구조를 보여주는 다이어그램

```python
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(14, 8))

# 애플리케이션 레이어
app_rect = mpatches.Rectangle((5, 6), 4, 1, facecolor='lightblue', edgecolor='black')
ax.add_patch(app_rect)
ax.text(7, 6.5, 'Application Layer', ha='center', va='center', fontsize=12, fontweight='bold')

# Redis
redis_rect = mpatches.Rectangle((1, 3), 2, 1, facecolor='lightcoral', edgecolor='black')
ax.add_patch(redis_rect)
ax.text(2, 3.5, 'Redis', ha='center', va='center', fontsize=11, fontweight='bold')
ax.text(2, 3.2, 'Rate Limiting\nCaching\nLocks', ha='center', va='top', fontsize=8)

# Kafka
kafka_rect = mpatches.Rectangle((7, 3), 2, 1, facecolor='lightgreen', edgecolor='black')
ax.add_patch(kafka_rect)
ax.text(8, 3.5, 'Kafka', ha='center', va='center', fontsize=11, fontweight='bold')
ax.text(8, 3.2, 'Event Streaming\nTask Queue', ha='center', va='top', fontsize=8)

# 연결선
ax.arrow(6, 6, -3, -2.5, head_width=0.2, head_length=0.2, fc='black', ec='black')
ax.arrow(8, 6, -1, -2.5, head_width=0.2, head_length=0.2, fc='black', ec='black')

ax.set_xlim(0, 10)
ax.set_ylim(2, 7)
ax.axis('off')
plt.title('분산 아키텍처 구조', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
```

**필요한 이미지**:
- 다이어그램: Application Layer가 상단에 있고, Redis와 Kafka가 하단에 배치
- Application에서 Redis와 Kafka로 화살표 연결
- 각 컴포넌트의 역할 표시
- 메시지 흐름 다이어그램 (선택적)


In [ ]:
from beanllm.infrastructure.distributed import (
    get_cache_stats,
    get_rate_limiter_stats,
    get_event_logger_stats,
    get_task_queue_stats,
)


async def monitoring_example():
    """분산 시스템 모니터링 예제"""

    print("🔍 분산 시스템 모니터링:\n")

    # 1. 캐시 통계
    print("📊 캐시 통계:")
    cache_stats = await get_cache_stats()
    print(f"  총 키 수: {cache_stats.get('total_keys', 0):,}")
    print(f"  캐시 히트: {cache_stats.get('hits', 0):,}")
    print(f"  캐시 미스: {cache_stats.get('misses', 0):,}")

    hit_rate = (
        cache_stats.get("hits", 0) / (cache_stats.get("hits", 0) + cache_stats.get("misses", 1))
        if cache_stats.get("hits", 0) + cache_stats.get("misses", 0) > 0
        else 0
    )
    print(f"  히트율: {hit_rate * 100:.1f}%")
    print(f"  메모리 사용: {cache_stats.get('memory_usage', 0) / 1024 / 1024:.1f} MB")
    print()

    # 2. Rate Limiter 통계
    print("📊 Rate Limiter 통계:")
    rate_limiter_stats = await get_rate_limiter_stats()
    print(f"  총 요청: {rate_limiter_stats.get('total_requests', 0):,}")
    print(f"  허용: {rate_limiter_stats.get('allowed', 0):,}")
    print(f"  거부: {rate_limiter_stats.get('rejected', 0):,}")

    rejection_rate = (
        rate_limiter_stats.get("rejected", 0) / rate_limiter_stats.get("total_requests", 1)
        if rate_limiter_stats.get("total_requests", 0) > 0
        else 0
    )
    print(f"  거부율: {rejection_rate * 100:.1f}%")
    print()

    # 3. Event Logger 통계
    print("📊 Event Logger 통계:")
    event_stats = await get_event_logger_stats()
    print(f"  총 이벤트: {event_stats.get('total_events', 0):,}")
    print(f"  성공: {event_stats.get('success', 0):,}")
    print(f"  실패: {event_stats.get('failed', 0):,}")
    print()

    if event_stats.get("by_type"):
        print("  이벤트 타입별:")
        for event_type, count in event_stats.get("by_type", {}).items():
            print(f"    - {event_type}: {count:,}")
        print()

    # 4. Task Queue 통계
    print("📊 Task Queue 통계:")
    queue_stats = await get_task_queue_stats()
    print(f"  대기 중: {queue_stats.get('pending', 0):,}")
    print(f"  처리 중: {queue_stats.get('processing', 0):,}")
    print(f"  완료: {queue_stats.get('completed', 0):,}")
    print(f"  실패: {queue_stats.get('failed', 0):,}")
    print()

    # 5. 전체 상태 요약
    print("📊 전체 시스템 상태:")

    # 건강 상태
    redis_ok = check_redis_health()
    kafka_ok = check_kafka_health()

    print(f"  Redis: {'🟢 정상' if redis_ok else '🔴 장애'}")
    print(f"  Kafka: {'🟢 정상' if kafka_ok else '🔴 장애'}")

    # 성능 지표
    print(
        f"  캐시 히트율: {hit_rate * 100:.1f}% {'🟢' if hit_rate > 0.8 else '🟡' if hit_rate > 0.5 else '🔴'}"
    )
    print(
        f"  Rate Limit 거부율: {rejection_rate * 100:.1f}% {'🟢' if rejection_rate < 0.1 else '🟡' if rejection_rate < 0.3 else '🔴'}"
    )

    # 큐 적체 상태
    queue_backlog = queue_stats.get("pending", 0) + queue_stats.get("processing", 0)
    print(
        f"  큐 적체: {queue_backlog:,} {'🟢' if queue_backlog < 100 else '🟡' if queue_backlog < 1000 else '🔴'}"
    )
    print()


# 실행
await monitoring_example()

print("💡 모니터링 포인트:")
print("  - 캐시 히트율 > 80%: 캐싱 효과적")
print("  - Rate Limit 거부율 < 10%: 적절한 제한")
print("  - 큐 적체 < 100: 처리 속도 충분")
print("  - 이벤트 실패율 < 1%: 시스템 안정")

print("\n💡 디버깅 방법:")
print("  1. 캐시 히트율 낮음 → TTL 증가 또는 키 설계 개선")
print("  2. Rate Limit 거부율 높음 → 제한 완화 또는 사용자 교육")
print("  3. 큐 적체 발생 → 워커 수 증가 또는 배치 크기 조정")
print("  4. 이벤트 실패 → Kafka 연결 확인 또는 Fallback 활성화")

print("\n💡 알림 설정 권장:")
print("  - 캐시 히트율 < 50%")
print("  - Rate Limit 거부율 > 30%")
print("  - 큐 적체 > 1000")
print("  - Redis/Kafka 연결 실패")

### 6.5 모니터링 & 디버깅

분산 시스템의 상태를 모니터링하고 디버깅하는 방법을 학습합니다.

In [ ]:
from beanllm.infrastructure.distributed import BatchProcessor
import time


async def batch_processing_example():
    """대용량 데이터 배치 처리 예제"""

    print("🔍 대용량 데이터 배치 처리:\n")

    # BatchProcessor 생성
    batch_processor = BatchProcessor(
        batch_size=10,  # 배치 크기
        max_workers=4,  # 워커 수
        use_distributed=True,  # 분산 큐 사용
    )

    # 처리할 문서 목록 (1000개)
    documents = [f"document_{i:04d}.pdf" for i in range(1000)]

    print(f"📚 총 문서 수: {len(documents)}")
    print(f"⚙️  배치 크기: {batch_processor.batch_size}")
    print(f"👷 워커 수: {batch_processor.max_workers}")
    print()

    # 처리 함수 정의
    async def process_document(doc_path: str) -> dict:
        """단일 문서 처리"""
        # 실제 처리 시뮬레이션
        await asyncio.sleep(0.05)  # 50ms 처리 시간

        return {
            "doc_path": doc_path,
            "status": "success",
            "num_pages": 10,
            "num_chunks": 25,
        }

    # 배치 처리 실행
    print("🔄 배치 처리 시작...")
    start = time.time()

    results = await batch_processor.process_batch(
        items=documents, process_func=process_document, show_progress=True  # 진행률 표시
    )

    elapsed = time.time() - start

    # 결과 분석
    success_count = sum(1 for r in results if r.get("status") == "success")
    total_pages = sum(r.get("num_pages", 0) for r in results)
    total_chunks = sum(r.get("num_chunks", 0) for r in results)

    print(f"\n✅ 배치 처리 완료!")
    print(f"   ⏱️  총 처리 시간: {elapsed:.2f}초")
    print(f"   📊 처리량: {len(documents) / elapsed:.1f} docs/sec")
    print(f"   ✅ 성공: {success_count}/{len(documents)}")
    print(f"   📄 총 페이지: {total_pages:,}")
    print(f"   📝 총 청크: {total_chunks:,}")
    print()

    # 순차 처리와 비교
    print("📊 순차 처리와 비교:")
    sequential_time = len(documents) * 0.05  # 순차 처리 예상 시간
    speedup = sequential_time / elapsed
    print(f"   순차 처리 예상 시간: {sequential_time:.2f}초")
    print(f"   배치 처리 시간: {elapsed:.2f}초")
    print(f"   속도 향상: {speedup:.1f}배")
    print()


# 실행
await batch_processing_example()

print("💡 BatchProcessor의 장점:")
print("  - 병렬 처리: CPU/IO 리소스 효율적 활용")
print("  - 분산 큐: 여러 워커로 작업 분산")
print("  - 진행률 표시: 실시간 진행 상황 확인")
print("  - 에러 핸들링: 개별 항목 실패해도 전체 계속")
print("  - 재시도: 실패한 항목 자동 재시도")

print("\n💡 사용 사례:")
print("  - 대량 문서 임베딩 생성 (1000+ 문서)")
print("  - 이미지 배치 처리 (OCR, Vision RAG)")
print("  - Knowledge Graph 대량 구축")
print("  - 데이터 마이그레이션")

### 6.4 대용량 데이터 처리 (BatchProcessor)

BatchProcessor를 사용하여 대용량 데이터를 효율적으로 처리합니다.

In [ ]:
from beanllm.infrastructure.distributed import (
    check_redis_health,
    check_kafka_health,
)


async def test_fallback():
    """Fallback 메커니즘 테스트"""

    print("🔍 Fallback 메커니즘 테스트:\n")

    # 1. 분산 컴포넌트 상태 확인
    print("📊 분산 컴포넌트 상태:")
    redis_ok = check_redis_health()
    kafka_ok = check_kafka_health()

    print(f"  Redis: {'✅ 연결됨' if redis_ok else '❌ 연결 실패 → Fallback'}")
    print(f"  Kafka: {'✅ 연결됨' if kafka_ok else '❌ 연결 실패 → Fallback'}")
    print()

    # 2. 분산 기능 사용 (자동 Fallback)
    @with_distributed_features(
        pipeline_type="test",
        enable_cache=True,
        enable_rate_limiting=True,
        enable_event_streaming=True,
    )
    async def test_with_fallback(query: str) -> str:
        return f"처리 완료: {query}"

    # 3. 테스트 실행
    print("🔄 테스트 실행:")
    try:
        result = await test_with_fallback("테스트 쿼리")
        print(f"✅ 성공: {result}")
        print()

        if not redis_ok:
            print("💡 Redis 연결 실패:")
            print("  - Rate Limiting → 인메모리 Rate Limiter 사용")
            print("  - Caching → 인메모리 캐시 사용")
            print("  - Distributed Lock → 로컬 락 사용")
            print()

        if not kafka_ok:
            print("💡 Kafka 연결 실패:")
            print("  - Event Streaming → 로컬 로그 사용")
            print("  - Task Queue → 인메모리 큐 사용")
            print()

        print("✅ Fallback이 정상 작동하여 기능은 계속 동작합니다!")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")

    print()


# 실행
await test_fallback()

print("💡 Fallback의 장점:")
print("  - 안정성: 분산 컴포넌트 실패해도 서비스 중단 없음")
print("  - 유연성: 개발 환경에서는 인메모리, 프로덕션에서는 분산")
print("  - 점진적 도입: 분산 컴포넌트를 하나씩 추가 가능")
print("  - 디버깅: 로컬 환경에서 쉽게 테스트")

print("\n💡 Fallback 동작 방식:")
print("  1. 분산 컴포넌트 연결 시도")
print("  2. 연결 실패 시 자동으로 인메모리로 전환")
print("  3. 경고 로그 출력 (운영자에게 알림)")
print("  4. 기능은 정상 작동 (성능 약간 저하)")

### 6.3 Fallback 메커니즘 테스트

분산 컴포넌트 실패 시 자동으로 인메모리로 전환되는 것을 확인합니다.

In [ ]:
import os
import time
import statistics


async def benchmark_performance(use_distributed: bool, num_requests: int = 20):
    """성능 벤치마크"""

    # 환경 설정
    os.environ["USE_DISTRIBUTED"] = "true" if use_distributed else "false"

    # 간단한 함수 정의
    @with_distributed_features(
        pipeline_type="test",
        enable_cache=True,
        enable_rate_limiting=False,  # 성능 비교를 위해 비활성화
        enable_event_streaming=False,
    )
    async def test_function(query: str) -> str:
        await asyncio.sleep(0.1)  # 처리 시뮬레이션
        return f"결과: {query}"

    latencies = []

    # 첫 번째 요청 (캐시 미스)
    start = time.time()
    await test_function("테스트 쿼리")
    first_latency = time.time() - start

    # 나머지 요청 (캐시 히트)
    for i in range(num_requests - 1):
        start = time.time()
        await test_function("테스트 쿼리")
        latency = time.time() - start
        latencies.append(latency)

    return {
        "mode": "분산 아키텍처" if use_distributed else "인메모리",
        "first_latency": first_latency,
        "avg_latency": statistics.mean(latencies),
        "p50_latency": statistics.median(latencies),
        "p95_latency": (
            statistics.quantiles(latencies, n=20)[18] if len(latencies) >= 20 else max(latencies)
        ),
        "total_requests": num_requests,
    }


print("🔍 성능 비교: 분산 vs 인메모리\n")

# 인메모리 벤치마크
print("📊 인메모리 모드:")
inmemory_result = await benchmark_performance(use_distributed=False, num_requests=20)
print(f"  첫 요청: {inmemory_result['first_latency']:.4f}초")
print(f"  평균: {inmemory_result['avg_latency']:.4f}초")
print(f"  P50: {inmemory_result['p50_latency']:.4f}초")
print(f"  P95: {inmemory_result['p95_latency']:.4f}초")
print()

# 분산 아키텍처 벤치마크
print("📊 분산 아키텍처 모드:")
distributed_result = await benchmark_performance(use_distributed=True, num_requests=20)
print(f"  첫 요청: {distributed_result['first_latency']:.4f}초")
print(f"  평균: {distributed_result['avg_latency']:.4f}초")
print(f"  P50: {distributed_result['p50_latency']:.4f}초")
print(f"  P95: {distributed_result['p95_latency']:.4f}초")
print()

# 비교 결과
print("📊 비교 결과:")
first_overhead = (
    (distributed_result["first_latency"] - inmemory_result["first_latency"])
    / inmemory_result["first_latency"]
    * 100
)
avg_overhead = (
    (distributed_result["avg_latency"] - inmemory_result["avg_latency"])
    / inmemory_result["avg_latency"]
    * 100
)

print(f"  첫 요청 오버헤드: {first_overhead:+.1f}%")
print(f"  평균 오버헤드: {avg_overhead:+.1f}%")
print()

print("💡 분석:")
print("  - 인메모리: 낮은 레이턴시, 단일 서버 제한")
print("  - 분산: 약간의 네트워크 오버헤드, 수평 확장 가능")
print("  - 캐시 히트 시: 두 방식 모두 빠른 응답")
print("  - 권장: 프로토타입은 인메모리, 프로덕션은 분산")

### 6.2 성능 비교: 분산 vs 인메모리

분산 아키텍처와 인메모리 방식의 성능을 비교합니다.

## 7. 다음 단계

Distributed Architecture를 마스터했습니다! 🎉

### 📚 전체 학습 완료

모든 튜토리얼을 완료하셨습니다! 이제 beanllm의 모든 기능을 활용할 수 있습니다.

### 🔗 관련 문서

- [README.md](../../README.md) - 프로젝트 개요
- [ARCHITECTURE.md](../../ARCHITECTURE.md) - 아키텍처 상세 설명
- [DEPENDENCY_RULES.md](../../DEPENDENCY_RULES.md) - 의존성 규칙
- [API Reference](../API_REFERENCE.md) - 완전한 API 문서

### 🚀 다음 단계

1. **실제 프로젝트에 적용**: 학습한 내용을 실제 프로젝트에 적용해보세요
2. **커뮤니티 참여**: GitHub에서 이슈를 제출하거나 PR을 제출하세요
3. **기능 확장**: 필요한 기능을 추가하거나 개선하세요

---

**✅ 모든 튜토리얼 완료! 축하합니다! 🎉**
